<h1 style="text-align:center;">Setting license key</h1>

<div class="alert alert-block alert-info"  style="border-radius: 20px;">
 Welcome to <b>python-iLOrest-library</b> <br> This is an example of <b>setting license key.</b> <br> As you are here let's get it done easily.<br>
    To perform this you need to have <a href = "https://github.com/HewlettPackard/python-ilorest-library" style = "background-color:white">python-ilorest-library</a> installed in your machine.<br>
    And please keep your username and password handy!
    <br>
    <b>Also do execute each successive cell once to avoid any errors!</b>
    
</div>

 <div class="alert alert-block alert-success"  style="border-radius: 20px;">
    <h3> 1. Below we are importing all the necessary modules from the ilorest library. </h3><br>
    <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys </a>
    </li>
    <li>json </li>
    <li> redfish </li>
    
</div>

In [ ]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

from get_resource_directory import get_resource_directory

<div class = "alert alert-block alert-success"  style="border-radius: 20px;"><h3>2. This is our set_license_key function overview.</h3>
    <br>
  There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to change the settings.

1. In the <b>set_license_key</b> function we are taking two parameters i.e. <b>_redfishobj</b> and <b>ilo_key</b>. In the <b>set_license_key</b> function we are first using <b>resource_instances</b> variable to obtain information about the resource directory by using get_resource_directory module.
<br>
    
2. In the iLO rest api the information about license key is stored inside the <b>managers json object</b>.
    Through managers_response variable  we are making a http <a href = "https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods" style="background-color:powderblue;">GET operation</a> to and trying to obtain the license service about iLO system.
    <br>

3. In the managers response object we are iterating on all the members with different <b>odata_id</b> .
    Then again an http GET request is made and response is stored in <b>managers_members_response</b>.
    The license service information is ultimately present in the <b>managers_members_response</b>.
    <br>

4. Else if resource directory is enabled then we will simply iterate through all instances inside resources 
    and if HpeiLOLicense type is present in <b>@odata.type</b> attribute then we will store information about license key in ilo_lic_uri and break out of iteration.
    <br>
5. <b>If ilo_lic_uri is not null</b> after making above operations, we can perform the required actions to set license key. 
    <ul>
    <li>The function sends a GET request to the iLO license service to retrieve the current license data. If the ConfirmationRequest field exists, it is printed to the console; otherwise, the LicenseKey field is printed.</li>
<li>The function sends a POST request to the iLO license service with the new license key provided in the ilo_key argument.</li>
    <li>The job of post request here is to set the license key at the provided location and send a response whether the job was done properly or not.</li>
    <li>If the response status is 400, the function attempts to print the extended error message provided by iLO. If the status is not 200 or 201, the function prints an error message to the console.</li>
<li>If the response status is 200, the function prints a success message to the console along with the response JSON.</li>
    </ul>

 </div>

In [ ]:
def set_license_key(_redfishobj, ilo_key):

    ilo_lic_uri = None

    resource_instances = get_resource_directory(_redfishobj)
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        managers_uri = _redfishobj.root.obj['Managers']['@odata.id']
        managers_response = _redfishobj.get(managers_uri)
        managers_members_uri = next(iter(managers_response.obj['Members']))['@odata.id']
        managers_members_response = _redfishobj.get(managers_members_uri)
        ilo_lic_uri = managers_members_response.obj.Oem.Hpe.Links['LicenseService']['@odata.id']
    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if '#HpeiLOLicense.' in instance['@odata.type']:
                ilo_lic_uri = instance['@odata.id']

    if ilo_lic_uri:
        ilo_license_collection = _redfishobj.get(ilo_lic_uri)
        ilo_license_member_uri = next(iter(ilo_license_collection.obj['Members']))['@odata.id']
        try:
            ilo_license_data = _redfishobj.get(ilo_license_member_uri).obj['ConfirmationRequest']\
                                                                                            ['EON']
        except KeyError:
            sys.stdout.write("This machine will not show the full License Key.\n")
            ilo_license_data = _redfishobj.get(ilo_license_member_uri).obj['LicenseKey']

        sys.stdout.write("Current iLO License Data:\n")
        print(json.dumps(ilo_license_data, indent=4, sort_keys=True))
        resp = _redfishobj.post(ilo_lic_uri, {'LicenseKey' : ilo_key})
        #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info
        #error message to see what went wrong
        if resp.status == 400:
            try:
                print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, \
                                                                                sort_keys=True))
                sys.stderr.write("Check the validity of your license key...\n")
            except Exception as excp:
                sys.stderr.write("A response error occurred, unable to access iLO " \
                                 "Extended Message Info...")
        elif resp.status != 200:
            sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
        else:
            print("Success!\n")
            print(json.dumps(resp.dict, indent=4, sort_keys=True))


<div class="alert alert-block alert-success"  style="border-radius: 20px;"><h3 >3. This is Main function overview.</h3><br>
<b>The Main function: </b> PLEASE ENTER YOU LOGIN CREDENTIALS HERE TO PROCEED FURTHER<br>
    <ul>
        <li>Once your credentials are entered a redfish object is created with the help of RedfishClient class and a session is created and now you will be logged in to the server.</li>
        <li>If server is down or unreachable we get server not reachable message.</li>
        <li>If not then we make a call to <b>set_license_key function</b> to set license key of system.</li>
        <li>After getting required information <i><b>Redfish.logout()</b></i> closses the particular session and logs us out of the system.</li>
    </ul>
</div>

In [ ]:
if __name__ == "__main__":
    # When running on the server locally use the following commented values
    #SYSTEM_URL = None
    #LOGIN_ACCOUNT = None
    #LOGIN_PASSWORD = None

    # When running remotely connect using the secured (https://) address,
    # account name, and password to send https requests
    # SYSTEM_URL acceptable examples:
    # "https://10.0.0.100"
    # "https://ilo.hostname"
    SYSTEM_URL = "https://10.0.0.100"
    LOGIN_ACCOUNT = "admin"
    LOGIN_PASSWORD = "password"

    # Must be a valid iLO License Key
    ILO_LICENSE_KEY = "XXXX-XXXX-XXXX-XXXX-XXXXX"
    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = True

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    set_license_key(REDFISHOBJ, ILO_LICENSE_KEY)
    REDFISHOBJ.logout()


<div class = "alert alert-block alert-warning"  style="border-radius: 20px;">
    <b>When running on the server locally use the following commented values</b><br>
 SYSTEM_URL = None<br>
 LOGIN_ACCOUNT = None<br>
 LOGIN_PASSWORD = None<br>

   <b>When running remotely</b> connect using the secured (https://) address,
     account name, and password to send https requests
    <br>
     SYSTEM_URL acceptable examples:<br>
     https://10.0.0.100<br>
     https://ilo.hostname
</div>

<div class = "alert alert-block alert-success"  style="border-radius: 20px;">In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
    
</div>